In [1]:
import pandas as pd
import glob
import os

# --- 1. Настройка путей ---

# Убедимся, что мы находимся в главной директории проекта
# (Этот код должен быть в начале вашего ноутбука)
PROJECT_PATH = "/content/drive/MyDrive/GitHub/no2_prediction_pipeline"
%cd {PROJECT_PATH}

# Папка, где лежат ваши разделенные по годам CSV-файлы
# Судя по скриншоту, это 'data/raw/add_predictors/'
SOURCE_FOLDER = 'data/raw/add_predictors'

# --- 2. Поиск и фильтрация нужных файлов ---

# Составляем шаблон для поиска всех CSV в указанной папке
all_csv_files = glob.glob(os.path.join(SOURCE_FOLDER, "*.csv"))

# Создаем пустой список, куда будем складывать датафреймы
files_to_combine = []

print("Найдены следующие файлы для объединения:")
for file_path in all_csv_files:
    # Отбираем только файлы за нужные годы (2019-2024)
    if any(str(year) in file_path for year in range(2019, 2025)):
        print(f" - {os.path.basename(file_path)}")
        files_to_combine.append(file_path)

# --- 3. Чтение и объединение ---

# Создаем список, в который прочитаем каждый CSV как отдельный DataFrame
list_of_dataframes = [pd.read_csv(file) for file in files_to_combine]

# Объединяем все датафреймы из списка в один большой
if list_of_dataframes:
    combined_df = pd.concat(list_of_dataframes, ignore_index=True)

    # --- 4. Постобработка и сохранение ---

    # Преобразуем колонку 'date' в правильный формат даты
    combined_df['date'] = pd.to_datetime(combined_df['date'])

    # Сортируем весь датафрейм по дате, чтобы все было в хронологическом порядке
    combined_df = combined_df.sort_values(by='date').reset_index(drop=True)

    print("\n--- Итоговый датафрейм ---")
    print(f"Общее количество строк: {len(combined_df)}")
    print(f"Временной диапазон: от {combined_df['date'].min().date()} до {combined_df['date'].max().date()}")
    print("Первые 5 строк объединенного файла:")
    print(combined_df.head())

    # Папка для сохранения обработанных данных (хорошая практика)
    PROCESSED_FOLDER = 'data/processed'
    if not os.path.exists(PROCESSED_FOLDER):
        os.makedirs(PROCESSED_FOLDER)

    # Сохраняем итоговый файл
    output_filename = os.path.join(PROCESSED_FOLDER, 'Surgut_data_2019_to_2024_COMBINED_WITH_AAI.csv')
    combined_df.to_csv(output_filename, index=False)

    print(f"\n✅ Файлы успешно объединены и сохранены в: {output_filename}")

else:
    print("❌ Не найдено файлов для объединения. Проверьте путь в SOURCE_FOLDER.")

/content/drive/MyDrive/GitHub/no2_prediction_pipeline
Найдены следующие файлы для объединения:
 - Surgut_data_2019_Feb-Sep_WITH_AAI.csv
 - Surgut_data_2020_Feb-Sep_WITH_AAI.csv
 - Surgut_data_2021_Feb-Sep_WITH_AAI.csv
 - Surgut_data_2022_Feb-Sep_WITH_AAI.csv
 - Surgut_data_2023_Feb-Sep_WITH_AAI.csv
 - Surgut_data_2024_Feb-Sep_WITH_AAI.csv

--- Итоговый датафрейм ---
Общее количество строк: 1454
Временной диапазон: от 2019-02-01 до 2024-09-30
Первые 5 строк объединенного файла:
        date    city  no2_trop_mean  aai_mean  cloud_frac_mean  \
0 2019-02-01  Surgut       0.000033 -0.875213         0.358617   
1 2019-02-02  Surgut       0.000033 -1.112367         0.234913   
2 2019-02-03  Surgut       0.000037 -1.398887         0.269149   
3 2019-02-04  Surgut       0.000030 -1.116266         0.371478   
4 2019-02-05  Surgut       0.000037 -0.972759         0.423403   

   temperature_celsius  pressure_hpa  u_wind_10m  v_wind_10m  
0           -41.978251   1038.040702   -2.045977    0.0241

In [3]:
combined_df

,date,city,no2_trop_mean,aai_mean,cloud_frac_mean,temperature_celsius,pressure_hpa,u_wind_10m,v_wind_10m
0,2019-02-01,Surgut,0.000033,-0.875213,0.358617,-41.978251,1038.040702,-2.045977,0.024103
1,2019-02-02,Surgut,0.000033,-1.112367,0.234913,-39.245128,1034.768421,-1.990192,-1.203328
2,2019-02-03,Surgut,0.000037,-1.398887,0.269149,-38.905555,1037.783115,-0.378380,-1.401940
3,2019-02-04,Surgut,0.000030,-1.116266,0.371478,-33.816355,1038.234460,0.179797,-1.944240
4,2019-02-05,Surgut,0.000037,-0.972759,0.423403,-33.523440,1041.622638,-2.200455,-0.714484
...,...,...,...,...,...,...,...,...,...
1449,2024-09-26,Surgut,NaN,-0.652925,0.655325,1.054706,1013.029982,2.535897,-1.753471
1450,2024-09-27,Surgut,NaN,0.000000,0.736774,2.934939,1014.392670,0.153333,2.872589
1451,2024-09-28,Surgut,0.000014,-0.177757,0.331710,9.839770,1013.596368,2.468348,3.017167
1452,2024-09-29,Surgut,NaN,0.000000,0.831001,11.902298,1007.221111,4.335900,1.906272
